## Normal estimation
### Numpy
ここでは、座標値のみを持つ点の点群から法線を推定します。  

法線の推定は、[PCLのEstimating Surface Normals in a PointCloud](https://pcl.readthedocs.io/projects/tutorials/en/pcl-1.12.0-rc1/normal_estimation.html)を参考に作成します。

$$
\mathcal{C}=\frac{1}{k} \sum_{i=1}^{k} \cdot\left(\boldsymbol{p}_{i}-\overline{\boldsymbol{p}}\right) \cdot\left(\boldsymbol{p}_{i}-\overline{\boldsymbol{p}}\right)^{T}, \mathcal{C} \cdot \overrightarrow{\mathbf{v}}_{j}=\lambda_{j} \cdot \overrightarrow{\mathbf{v}_{j}}, j \in\{0,1,2\}
$$

このsubsectionでは以下のパッケージを使用します。


In [1]:
%load_ext autoreload
%autoreload 2

In [46]:
# for normal estimation
import numpy as np
from tutlibs.normals import normal_estimation

# for description
from tutlibs.io import Points as io
from tutlibs.transformation import translation
from tutlibs.visualization import Points as visualizer

法線推定は以下のとおりです。

In [64]:
# Load a point cloud
xyz, colors, data = io.read('../data/bunny_pc.ply')

# Estimate normals
k = 15
estimated_normals = normal_estimation(xyz, k=k)

# Get GT normals
gt_normals = np.stack([data['nx'], data['ny'], data['nz']], axis=-1)
gt_xyz = translation(xyz, np.array([1, 0, 0]))

# Visualize normals
visualizer.k3d(
    np.vstack([xyz, gt_xyz]), 
    np.vstack([estimated_normals, gt_normals]),
    color_range=[-1, 1]
)

Output()

上記の推定では、x軸方向に0~1の間にある点群が法線推定を施した点群(法線推定点群)、1~2の間にある点群がMeshから直接取得した法線を持つ点群(GT法線点群)です。法線は法線マップと呼ばれる色合いによって示されています。法線マップはなだらかな曲線を持つ表面に対しては徐々に色が変わり、そうでない表面では色が著しく変わります。上記のGT法線点群では曲線に沿って色が変化しているのに対し、法線推定点群はまだら模様になっています。これは、normal estimation時に点群の表裏を考慮していないことが原因です。

法線情報は

### Use of Open3D
ここで使用するパッケージは以下のとおりです。

In [ ]:
import open3d as o3d
pc = o3d.io.read_point_cloud('../data/bunny_pc.ply')
pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals = np.asarray(pc.normals)

normals_colors = (normals + 1) / 2
xyz = np.asarray(pc.points)
visualizer.k3d(xyz, normals_colors)
normals_colors = normals_colors * 255

os.makedirs('outputs/', exist_ok=True)
io.write('outputs/o3d_normal.ply', xyz, normals_colors)

Output()